In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import _pickle as pkl
from numpy import *
from sklearn import metrics
from sklearn import metrics


### 挑选了LDL里面的aa_knn，较为简单，不知道会不会影响结果

In [2]:
def aa_knn(trainFeature, trainLabel, testFeature, testLabel, k):
    rows = testFeature.shape[0]
    cols = testFeature.shape[1]

    train_num = trainFeature.shape[0]
    dist = np.zeros(shape=(train_num))
    prediction = np.zeros(shape=(rows,trainLabel.shape[1]))

    for i in range(rows):
        test = testFeature[i,:]
        for j in range(train_num):
            train = trainFeature[j,:]
            V = test - train
            sqV = V**2
            sqDis = sqV.sum(axis=0)
            dist[j] = math.sqrt(sqDis)

        indice = dist.argsort()
        sort_dis = sorted(dist)
        sum_label = np.zeros(shape=trainLabel.shape[1])
        for l in range(k):
            sum_label = sum_label + trainLabel[l,:]

        prediction[i,:] = sum_label/k
    
    idx = prediction.argmax(axis=1)
    prediction = (idx[:,None] == np.arange(prediction.shape[1])).astype(int)

    score = metrics.accuracy_score(prediction, testLabel)

    return prediction, score 

In [41]:
file_label = open("ldlLabelAll.txt",'r')

label = []

for line in file_label:
    one_label = list(map(lambda x : float(x), line.split()))
    label.append(one_label)

In [44]:
idxs = np.random.permutation(8)

x_ = trainLabel[idxs[0]]
for a, idx in enumerate(idxs):
    if a == 0:
        print(0)
    else:
        x_ = np.vstack((x_,trainLabel[idx]))
    

0


In [45]:
print(x_)

[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [69]:
def tmc_shap(tolerance, trainFeature, trainLabel, testFeature, testLabel, k, iterations, sources=None):
    if sources is None:
        sources = {i: np.array([i]) for i in range(len(trainFeature))}
    elif not isinstance(sources, dict):
        sources = {i: np.where(sources == i)[0] for i in set(sources)}

    marginals = []
    mem_tmc = np.zeros((0,len(trainFeature)))

    for i in range(iterations):

        idxs = np.random.permutation(len(sources))
        marginal_contribs = np.zeros(len(trainFeature))

        #x_batch = np.zeros((0,) + tuple(trainFeature.shape[1:]))
        #y_batch = np.zeros(0, int)
        x_batch = trainFeature[idxs[0]]
        y_batch = trainLabel[idxs[0]]

        truncation_counter = 0

        #源代码里面是随机抽取测试数据，进行100次实验获得到的平均socre，这里直接设置为零
        new_score = 0

        
        for sh, idx in enumerate(idxs):
            old_score = new_score

            if sh == 0 :
                a=0
            else:
                x_batch = np.vstack((x_batch, trainFeature[idx]))
                y_batch = np.vstack((y_batch, trainLabel[idx]))

            if x_batch.shape[0] >= 2:
                _, new_score = aa_knn(trainFeature, trainLabel, testFeature, testLabel, k)

            marginal_contribs[sources[idx]] = (new_score - old_score)/len(sources[idx])
            mean_score = 0.18

            distance_to_full_score = np.abs(new_score - mean_score)
            if distance_to_full_score <= tolerance * mean_score:
                truncation_counter += 1
                if truncation_counter > 5:
                    break
            else:
                truncation_counter = 0
        
        mem_tmc = np.concatenate([mem_tmc, np.reshape(marginal_contribs, (1,-1))])

    value_tmc = np.mean(mem_tmc,0)

    return value_tmc,mem_tmc


In [ ]:
 """
        for sh, idx in enumerate(idxs):
            old_score = new_score

            if sh == 0 :
                a=0
            else:


                x_batch = np.vstack((x_batch, trainFeature[idx]))
                y_batch = np.vstack((y_batch, trainLabel[idx]))
"""


In [4]:
file_feature = open("ldlFeature.txt",'r')
feature = []

for line in file_feature:
    one_fea = list(map(lambda x : float(x), line.split()))
    feature.append(one_fea)


file_label = open("binaryLabel.txt",'r')
label = []

for line in file_label:
    one_label = list(map(lambda x : float(x), line.split()))
    label.append(one_label)

In [5]:
trainFeature = array(feature[:70])
trainLabel = array(label[:70])

testFeature = array(feature[180:])
testLabel = array(label[180:])

In [70]:
shap,tmc = tmc_shap(0.3, trainFeature, trainLabel, testFeature, testLabel, 5, 20, sources=None)

In [73]:
print(len(tmc))
print(shap)

20
[0.03333333 0.         0.         0.         0.         0.03333333
 0.01666667 0.         0.         0.         0.         0.01666667
 0.         0.         0.01666667 0.         0.         0.
 0.         0.         0.         0.01666667 0.         0.03333333
 0.         0.01666667 0.         0.         0.         0.01666667
 0.01666667 0.         0.01666667 0.01666667 0.         0.
 0.         0.         0.01666667 0.03333333 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.01666667 0.         0.
 0.         0.         0.         0.01666667]


In [36]:
x_b = np.vstack((trainLabel[1], trainLabel[2]))
print(x_b)
x_c = np.vstack((x_b,trainLabel[3]))
print(x_c)

[[1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]]


In [16]:
a = np.array([[1, 2], [3, 4]])
x = np.zeros((0,a.shape[1]))

print(a)

[[1 2]
 [3 4]]


In [7]:
def one_iteration(tolerance, trainFeature, trainLabel, testFeature, testLabel, k, sources=None):
    if sources is None:
        sources = {i: np.array([i]) for i in range(len(trainFeature))}
    elif not isinstance(sources, dict):
        sources = {i: np.where(sources == i)[0] for i in set(sources)}

    idxs = np.random.permutation(len(sources))
    marginal_contribs = np.zeros(len(trainFeature))

    x_batch = np.zeros((0,trainFeature.shape[1]))
    y_batch = np.zeros(0,float)

    truncation_counter = 0

    #源代码里面是随机抽取测试数据，进行100次实验获得到的平均socre，这里直接设置为零
    new_score = 0

    for _, idx in enumerate(idx):
        old_score = new_score

        x_batch = np.concatenate(x_batch, trainLabel[sources[idx]])
        y_batch = np.concatenate(y_batch, trainFeature[sources[idx]])

        if len(set(y_batch)) >= 2:
            new_score = aa_knn()

        marginal_contribs[sources[idx]] = (new_score - old_score)/len(sources[idx])
        mean_score = 0.18

        distance_to_full_score = np.abs(new_score - mean_score)
        if distance_to_full_score <= tolerance * mean_score:
            truncation_counter += 1
            if truncation_counter > 5:
                break
        else:
            truncation_counter = 0
    return marginal_contribs
